# PIVpy getting started notebook:

This notebook shows an example of how one can use vecpy in order to load manipulate and display analyzed PIV data. 

### step 1 - import pivpy and dependencies

here we import the package code so that we can use it next 

In [ ]:
import os, sys 
# sys.path.append(os.path.abspath('../'))

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

from pivpy import io, pivpy, graphics

# for the sake of this tutorial, ignore warnings
# import warnings
# warnings.filterwarnings('ignore')

### step 2 - load the tests data

In order to load the data, first we need to set up the path to the data directory. Following that we need to get a list of files names that we would like to view/analyze. Finally we very quickly load the data in to a list of vec instances.

In [ ]:
# pointer to the directory with the data
import importlib.resources as importlib_resources
path_to_data  = importlib_resources.files('pivpy') / 'data'
# list the directory
# os.listdir(path_to_data)

In [ ]:
# let's read only the files from the Run* 
data = io.load_directory(path_to_data / 'Insight') # you can add also: basename='day2a*',ext='.vec')

In [ ]:
# let's check if it's read:
data.attrs['files']

In [ ]:
plt.quiver(data.x, data.y, data.u.isel(t=0), data.v.isel(t=0),scale=1)

### step 3 - plot some arrows

first things first - show a quiver plot

In [ ]:
fig, ax = graphics.quiver(data.isel(t=0), nthArr=2, arrScale=20, add_guide = False)

In [ ]:
fig, ax = graphics.quiver(data.isel(t=0), nthArr=3, arrScale=5)

In [ ]:
tmp = data.isel(t=0)
plt.quiver(tmp.x,tmp.y,tmp.u.T,tmp.v.T,scale=1)

In [ ]:
# we can read also a single file only into a 1 frame dataset
d = io.load_vec(path_to_data / 'Insight'/ 'Run000001.T000.D000.P000.H001.L.vec' )

In [ ]:
graphics.quiver(d.isel(t=0),arrScale=10, add_guide = False)

In [ ]:
d.isel(t=0).differentiate(coord='x').differentiate(coord='y')['u'].plot.pcolormesh()

and a vorticity map

In [ ]:
# prepare vorticity
d.piv.vec2scal('curl') # it will appear as d['w'] variable, 'w' for all scalar properties

# plot
fig, ax = graphics.contour_plot(d)

Also, velocity histograms in x and y directions

In [ ]:
fig, ax = graphics.histogram(data, normed = True)

We can also plot a whole list of vec's as subplots:

In [ ]:
fig, ax = graphics.quiver(data.isel(t=0), nthArr=4, arrScale=10)
fig.set_size_inches(10, 6)

# Last but not least - manipulation

lets create a linear combinatino of our data and then see how to manipulate the coordinate system

__Addition and Scalar multiplication__

In [ ]:
v = (data + 3*data - 2 * data.isel(t=0)) / 3.
graphics.quiver(v.isel(t=-1), arrScale=10)

__Crop__

In [ ]:
v = v.piv.crop([5,15,-5,-15])  #(xmin, xmax, ymin, ymax)
graphics.quiver(v.isel(t=-1), arrScale=10)

__Rotate__

In [ ]:
# v.piv.rotate(90) # not implemented

__Translation of Coordinate System__

In [ ]:
# we can also use some default plot from xarray
data.piv.vorticity()
data.isel(t=0)['w'].plot(robust=True)

In [ ]:
# low level quiver
plt.figure(figsize=(8,6))
plt.quiver(data.x,data.y,data.u[:,:,0], -data.v[:,:,0] ,data.u[:,:,0]**2 + data.v[:,:,0]**2,scale=.75)
plt.gca().invert_yaxis()

In [ ]:
test = io.create_sample_field(rows=25,cols=5)

In [ ]:
graphics.quiver(test,arrScale=5,aspectratio='auto')

In [ ]:
data = io.load_vec(path_to_data / 'openpiv_vec' / 'exp1_001_b.vec')

In [ ]:
variables,units,rows,cols, dt, frame, method = io.parse_header(path_to_data / 'openpiv_vec' / 'exp1_001_b.vec')
variables,units,rows,cols, dt, frame, method

In [ ]:
data.piv.quiver()

In [ ]:
data = io.load_directory(path_to_data / 'urban_canopy', ext='.vc7')
# vc7 files are for some reason need transpose
data['u'] = data['u'].transpose()
data['v'] = data['v'].transpose()

In [ ]:
data.isel(t=0).piv.quiver(arrScale=15,colorbar=True)
plt.gcf().set_size_inches(12,10)
#plt.gca().invert_yaxis()

In [ ]:
%pip install pyarrow


In [ ]:
%time
df = data.to_dataframe()
try:
    df.to_parquet('tmp.pq')
    print('Wrote tmp.pq')
except ImportError as e:
    print('Parquet engine not installed; skipping to_parquet:', e)


In [ ]:
%pip install netcdf4

In [ ]:
%time
data.to_netcdf('tmp.nc')

In [ ]:
tmp = xr.load_dataset('tmp.nc')
assert (tmp == data)

In [ ]:
ds = xr.Dataset.from_dataframe(df)
df.head(), ds.head()